In [128]:
import pandas as pd

In [129]:
# Считываем исходную таблицу ингредиентов из файла.
ingredients = pd.read_csv('./ingr_df.csv')

In [130]:
ingredients. rename(columns = {'ingr_id':'id_ingr', 
'ingr_name':'name_ingr'}, inplace = True )

In [131]:
# Считываем исходную таблицу рецептов из файла и формируем ее под структуру db
set_ingredients = pd.read_csv('./recipe_df.csv')
set_ingredients.drop(['cuisine','meal_mode'], axis=1, inplace=True)
set_ingredients. rename(columns = {'recipe_name':'name_recipe',
        'recipe_id':'id_recipe',                            
        'text':'text_recipe',
        'quantity':'qty',
        'measure':'unit',
        'ingr_qwn':'qty_string',
        'proteinContent':'protein',
        'fatContent':'fat',
        'carbohydrateContent':'carbo'
        }, inplace = True )

In [132]:
# Заменяем ingr_name в таблице set_ingredients на id_ingr из таблицы ingredients
mapper = dict(zip(ingredients.name_ingr, ingredients.id_ingr))
set_ingredients['id_ingr'] = set_ingredients.ingr_name.apply(lambda x: mapper[x])
set_ingredients.drop(['ingr_name'], axis=1, inplace=True)
# Формируем столбец id в таблице set_ingredients из индекса, это будет PK всей db
set_ingredients['id'] = list(set_ingredients.index)

In [133]:
def get_unit(un_list):
    """
    Функция формирования названия единиц измерения.
        Args:
            un_list(list): список слов, 
            начиная со второго в описании ингредиента
            
        Returns:
            unit(str): название единицы измерения
            в именительном падеже единственного числа
        """
    # Словарь замены падежей для единиц измерения ингредиентов
    subst={
            'туки' : 'тука',
            'ожек' : 'ожка',
            'ожки' : 'ожка',
            'штук' : 'штука',
            'чика' : 'чик',
            'уска' : 'усок',
            'иков' : 'ик',
            'кана' : 'кан',
            'анки' : 'анка',
            'сков' : 'сок',
            'ебля' : 'ебель',
            'овки' : 'овка',
            'блей' : 'бель',
            'чков' : 'чок',
            'анов' : 'ан',
            'овок' : 'овка',
            'анок' : 'анка',
            'учка' : 'учок',
            'овых' : 'овая',
            'овые' : 'овая',
            'йных' : 'йная',
            'йные' : 'йная',
            'ьших' : 'ьшой/большая'
             }
    ul=[]
    for word in un_list:
        
        if len(word)>3:
            if word[-4:] in subst.keys():
                subst_key = word[-4:]
                word = word[:-4]+subst[subst_key]
            
        ul.append(word)
    unit = (' ').join(ul)
    return unit

In [134]:
# Разбиваем описание ингредиента на список слов
set_ingredients['list'] = set_ingredients.qty_string.str.split(' ')
# Формируем столбец begin из первых слов описания ингредиента
set_ingredients['begin'] = set_ingredients.list.apply(lambda x: x[0])
# Формируем столбец end из списка остальных слов описания ингредиента
set_ingredients['end'] = set_ingredients.list.apply(lambda x: x[1:])
# Заменяем символы дробей в столбце begin из первых слов описания ингредиента
set_ingredients.begin = set_ingredients.begin.str.replace('½','0.5').replace('¼','0.25').replace(
    '⅓','0.33').replace('⅔','0.67').replace('¾','0.75').replace(',','.', regex=True)
# Формируем признак isalpha - первые слов описания ингредиента состоят из букв
set_ingredients['isalpha'] = set_ingredients.begin.str.isalpha()
# Формируем признак unit - названия единиц измерения
set_ingredients['unit'] = set_ingredients['end'].apply(get_unit)
# Формируем признак qty - количество ингредиента
set_ingredients['qty'] = set_ingredients.apply(lambda row : float(row.begin) if row.isalpha==False else 0, axis = 1)
# Уточняем признак unit: для текстовых названий единиц измерения используем всю исходную строку
set_ingredients['unit'] = set_ingredients.apply(lambda row: row.qty_string if row.qty==0 else row.unit, axis = 1)
# Удаление временных признаков
set_ingredients.drop(['list', 'begin', 'end', 'isalpha'], axis=1, inplace=True)

In [135]:
# Создаем таблицу units
units = pd.DataFrame(set_ingredients.unit.unique(), columns=['name'])
units['id_unit'] = list(units.index)

In [136]:
# Заменяем названия ингредиентов в таблице set_ingredients на их id
mapper = dict(zip(units.name,units.id_unit))
set_ingredients.unit = set_ingredients.unit.apply(lambda x: mapper[x])

In [137]:
# Создаем таблицу recipes
recipes = set_ingredients.copy(deep=True)
recipes.drop(['qty_string', 'unit', 'qty', 'id', 'id_ingr'], axis=1, inplace=True)
recipes=recipes.groupby(by='id_recipe').first().reset_index()
recipes['vegeterian'] = False

In [138]:
# После создания таблицы recipes удаляем лишние признаки из таблицы set_ingredients
set_ingredients.drop(['name_recipe', 'portions', 'calories', 'protein', 'fat', 'carbo', 'text_recipe'], axis=1, inplace=True)

In [ ]:
# Запись в файлы
set_ingredients.to_csv('./set_ingredients.csv', index=False)
recipes.to_csv('./recipes.csv', index=False)
units.to_csv('./units.csv', index=False)
ingredients.to_csv('./ingredients.csv', index=False)

### Визуализация таблиц

In [139]:
set_ingredients.sample(20)

,id_recipe,qty_string,unit,qty,id_ingr,id
4762,547,30 г,0,30.0,15,4762
5094,589,1 штука,1,1.0,19,5094
4477,509,100 мл,7,100.0,621,4477
3193,360,½ штуки,1,0.5,19,3193
1413,148,200 г,0,200.0,34,1413
5897,682,1 стакан,3,1.0,3,5897
2894,323,1 штука,1,1.0,40,2894
2744,304,60 мл,7,60.0,6,2744
294,31,1 пучок,10,1.0,144,294
4518,514,200 г,0,200.0,23,4518


In [140]:
recipes

,id_recipe,name_recipe,portions,calories,protein,fat,carbo,text_recipe,vegeterian
0,0,Сырники из творога,2,938,42,61.0,56.0,Положите весь творог в кастрюльку и разомните ...,False
1,1,Классическая шарлотка,12,217,5,5.0,38.0,Разогреть духовку. Отделить белки от желтков. ...,False
2,2,Свинина с карамелизованной морковью,4,551,39,32.0,28.0,Свинину зачистить и обсушить. Зубчики чеснока ...,False
3,3,Брауни (brownie),6,676,10,46.0,55.0,Шоколад разломать на кусочки и вместе со сливо...,False
4,4,Лазанья классическая с мясом,6,965,50,73.0,24.0,В сотейник положить сливочное масло и 2 ложки ...,False
...,...,...,...,...,...,...,...,...,...
738,758,Голубцы,4,541,20,35.0,37.0,Лук нарезать небольшим кубиком и обжарить на р...,False
739,759,Ленивые голубцы с рисом,4,680,33,37.0,54.0,"Пропустить через мясорубку капусту, морковь, л...",False
740,760,Эклеры со сливочно-творожным кремом,8,364,11,18.0,41.0,Воду вскипятить со сливочным маслом и солью. Н...,False
741,761,"Курица с грибами, запеченная под картофельно-с...",4,460,32,25.0,30.0,Включить духовку на 180 градусов. Картофель по...,False


In [141]:
units

,name,id_unit
0,г,0
1,штука,1
2,столовая ложка,2
3,стакан,3
4,чайная ложка,4
5,зубчик,5
6,по вкусу,6
7,мл,7
8,головка,8
9,кусок,9


In [142]:
ingredients

,id_ingr,name_ingr
0,0,Творог
1,1,Куриное яйцо
2,2,Пшеничная мука
3,3,Сахар
4,4,Подсолнечное масло
...,...,...
754,754,Чабер
755,755,Паста каннеллони
756,756,Мороженое
757,757,Сливочный ликер
